# The Experiment Container

This notebook explains how the database works as an experiment container.

In [1]:
import os

import qcodes as qc
from qcodes.dataset.sqlite.database import initialise_or_create_database_at

In [2]:
db_file_path = os.path.join(os.getcwd(), 'exp_container_tutorial.db')
initialise_or_create_database_at(db_file_path)

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]
Upgrading database; v1 -> v2: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 333.17it/s]
Upgrading database; v2 -> v3: : 0it [00:00, ?it/s]
Upgrading database; v3 -> v4: : 0it [00:00, ?it/s]
Upgrading database; v4 -> v5: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 334.18it/s]
Upgrading database; v5 -> v6: : 0it [00:00, ?it/s]
Upgrading database; v8 -> v9: 100%|█████████████████████████████████████████████████████| 1/1 [00:00<00:00, 247.69it/s]


## The experiments inside the database

The database holds a certain number of **experiments**. They may be viewed:

In [3]:
from qcodes.dataset.experiment_container import experiments
from qcodes.dataset.experiment_settings import get_active_experiment_id

In [4]:
experiments()

[]

Not surprisingly, our new database is empty.

We now introduce a term that we call it **active experiment**. In short, the active experiment is the experiment that is used when a QCoDeS `DataSet` is generated, if the user do not explicitly pass an experiment into objects that create that `DataSet`. So, that `DataSet` will belong to the experiment that is active. We do not want to go into the details of `DataSet` here, and refer to the [DataSet notebook](https://qcodes.github.io/Qcodes/examples/DataSet/DataSet-class-walkthrough.html) and [Performing measurements using qcodes parameters and dataset](https://qcodes.github.io/Qcodes/examples/DataSet/Performing-measurements-using-qcodes-parameters-and-dataset.html) for what we mean from `DataSet` and explicitly passing of an experiment.

By default, `get_active_experiment_id()` will return the maximum **exp_id** in the database, if you load a database already has experiment(s) in it. If the database has no experiment and you have not created or loaded any experiment, the return value will be `None`: 

In [5]:
get_active_experiment_id()

Let us add some experiments to explore more:

In [6]:
from qcodes.dataset.experiment_container import new_experiment, load_or_create_experiment

In [7]:
exp_a = new_experiment('first_exp', sample_name='old_sample')
exp_b = new_experiment('second_exp', sample_name='slightly_newer_sample')
# A more convenient function that can load an experiment
# OR create a new one if it does not exist:
exp_c = load_or_create_experiment('third_exp', sample_name='brand_new_sample')

In [8]:
experiments()

[first_exp#old_sample#1@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------,
 second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------------------,
 third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
 -----------------------------------------------------------------------------------]

We notice that each experiment is labelled by an integer number, which is the `exp_id` that we mentioned above. This ID can be used when looking up properties of each experiment.

Let's check again which experiment is active now:

In [9]:
get_active_experiment_id()

3

The latest created or loaded experiment in the kernel becomes the active experiment and the function returns the `exp_id` of that experiment.

Let us add some `DataSet` to our experiments. For the sake of clarity, we don't add any data to the `DataSet` here, and refer to the above-mentioned notebooks for the details. Note that the `new_data_set` function is used here ONLY for the sake of exercise and should NOT be used in the actual experiment.

In [10]:
from qcodes.dataset.data_set import new_data_set

In [11]:
new_data_set('run_a')

run_a #1@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------

Since the active experiment is exp_a (`exp_id`=3), the above `DataSet` belongs to this experiment.

In [12]:
exp_c

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------
1-run_a-1--0

Let's activate another experiment, and add a few runs to it. We know that the latest created/ loaded experiment will be the active experiment, so let's load exp_b:

In [13]:
from qcodes.dataset.experiment_container import load_experiment_by_name, load_experiment

In [14]:
load_or_create_experiment('second_exp', sample_name='slightly_newer_sample')

second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------------

Let's confirm that actually the second experiment (`exp_id` = 2) is active:

In [15]:
get_active_experiment_id()

2

In [16]:
new_data_set('first_run_b')

first_run_b #2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------------

In [17]:
new_data_set('second_run_b')

second_run_b #3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
----------------------------------------------------------------------

Let's check the content of this experiment:

In [18]:
exp_b

second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------------
2-first_run_b-1--0
3-second_run_b-2--0

We can also explicitly use `exp_id` in creating `DataSet`s, so let's add a `DataSet` to the first experiment:

In [19]:
new_data_set('first_run', exp_id=1)

first_run #4@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-------------------------------------------------------------------

Here, we want to cover a few possible cases related to active experiment. First, we utilize the following function to disable active experiment in our Python session:

In [20]:
from qcodes.dataset.experiment_settings import reset_active_experiment_id

In [21]:
reset_active_experiment_id()

Users may not need to use this reset function, but it enables us to demonstrate an important behavior here. Disabling the active experiment means the following should return `None`:

In [22]:
get_active_experiment_id()

If there is no active experiment, then it will be an error to create a `DataSet`. In a real use case, this happens if you initialize an empty database, and not start an experiment before creating a `DataSet`. In such a scenario, an error will appear asking you to start an experiment.
But, we know that if we initialize a database already has experiments, the active experiment will be the experiment with the maximum `exp_id`. Since we filled our database with a few experiments, we load it again to confirm this: 

In [24]:
initialise_or_create_database_at(db_file_path)

In [25]:
get_active_experiment_id()

3

As expected, right?

Then, any new `DataSet` should be assigned to exp_c with `exp_id`=3. Let's check it:

In [26]:
new_data_set('default_run')

default_run #5@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
---------------------------------------------------------------------

In [27]:
exp_c

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------
1-run_a-1--0
5-default_run-2--0

And the last `DataSet` with the name of 'default_run' is added to exp_c.

There are a few other useful functions to load experiments:

In [28]:
from qcodes.dataset.experiment_container import load_experiment, load_last_experiment, load_experiment_by_name

In [29]:
load_experiment(exp_id=1) # loads using exp_id

first_exp#old_sample#1@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------
4-first_run-1--0

In [30]:
load_experiment_by_name('second_exp', sample='slightly_newer_sample')  # loads using name and sample

second_exp#slightly_newer_sample#2@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------------
2-first_run_b-1--0
3-second_run_b-2--0

In [31]:
load_last_experiment() # loads the last experiment in the database

third_exp#brand_new_sample#3@C:\Users\a-fbonabi\temp_work\exp_container_tutorial.db
-----------------------------------------------------------------------------------
1-run_a-1--0
5-default_run-2--0